In [ ]:
%load_ext autoreload
%autoreload 2

# Ideal Material experiment

In [ ]:
import dolfinx as dfx

import h5py

%matplotlib widget
from matplotlib import pyplot as plt

from mpi4py import MPI

import numpy as np

from pathlib import Path

from ideal_material_const_current import free_energy, experiment

from cahn_hilliard_utils import AnalyzeOCP, Simulation

from plotting_utils import (
    dfx_spherical_mesh, 
    plot_time_sequence, 
    plot_charging_cycle, 
    PyvistaAnimation)

## Setup and run the Simulation

In [ ]:
comm_world = MPI.COMM_WORLD

mesh = dfx.mesh.create_unit_interval(comm_world, 128)

I_charge = 1.

rt_analysis = AnalyzeOCP(free_energy=free_energy)

sim = Simulation(
    mesh,
    free_energy=free_energy,
    T_final=2. / I_charge,
    I=I_charge,
    experiment=experiment,
    output_file="simulation_output/ideal_material/reference.xdmf",
    runtime_analysis=rt_analysis,
    gamma=0.0,
)

In [ ]:
sim.run()

## Read back the data

In [ ]:
with h5py.File("simulation_output/ideal_material/reference.h5", 'r+') as f:
    print(f.keys())

    # grid coordinates
    x_data = f["Mesh/mesh/geometry"][()]

    # time steps (convert from string to float)
    t = [float(t.replace("_", ".")) for t in f["Function/comp_0"].keys()]

    # list of data stored as numpy arrays
    u_data = [(f["Function/comp_0"][u_key][()].squeeze(),
               f["Function/comp_1"][u_key][()].squeeze())
              for u_key in f["Function/comp_0"]]

u_data = np.array(u_data)

## Visualizations

In [ ]:
plot_time_sequence((x_data, t, u_data), lambda y: y, plot_mu=False)
# plot_time_sequence(sim.output, lambda y: np.exp(y) / (np.exp(y) + 1))

In [ ]:
mesh_3d, _, _ = dfx_spherical_mesh(comm_world, resolution=0.1, optimize=False)

In [ ]:
output = sim.output

anim = PyvistaAnimation(
    sim.output,
    # rt_analysis,
    mesh_3d=mesh_3d,
    c_of_y=lambda y: np.exp(y) / (1 + np.exp(y)),
    res=1.0,
    clim=[0.0, 1.0],
    cmap="hot",
)

widget = anim.get_slider_widget()

In [ ]:
pp_dir = Path("../post_processing/")
exp_dir = Path("const_current")
material_dir = Path("ideal_material")

output_path = pp_dir / material_dir / exp_dir
output_path.mkdir(exist_ok=True, parents=True)

anim.get_gif_animation(output_path / "anim.gif")

In [ ]:
q, f_bar, mu_bc = np.array(rt_analysis.data).T

eps = 1e-3

chart = plot_charging_cycle([(I_charge, q, mu_bc)], lambda q: np.log(q / (1 - q)))